# This notebook checks the fit function parameters and phase differences.

In [2]:
import numpy as np
import os.path

HOME_DIR = os.path.expanduser("~")
ES_BASEDIR = HOME_DIR + "/Dropbox/Research/energy_spectra"
day = "151204"
prefix = "GX339-BQPO"

In [3]:
def fit_function(t, p):
    """
    Computing a function to fit to the spectral parameter variations. This is
    exactly the same as what's in energy_spectra/multifit_plots.py and
    simulate/fake_qpo_spectra.py.

    Parameters
    ----------
    t : np.array of floats
        1-D array of time steps for the fit function.

    p : np.array of floats
        1-D array of the function parameters.

    Returns
    -------
    np.array of floats
        1-D array of the function fit to the data at steps t with parameters p.
    """
    return p[0] * np.sin(2. * np.pi * t + p[1]) + \
           p[2] * np.sin(2. * 2. * np.pi * t + p[3]) + p[4]

In [8]:
def get_funcfit_pars(fit_specifier, third):
    
    boot_parfit_file = ES_BASEDIR + "/out_es/" + prefix + \
            "/bootstrapped/" + prefix + "_" + day + "_" + \
            fit_specifier + "_funcfit.txt"    
    assert os.path.isfile(boot_parfit_file)
    
    t_bins = np.arange(0,1,0.001)
    all_boot_bestfits = np.zeros((3, 5, 1))
    all_max_phase_boot = np.zeros((3,1))
    all_min_phase_boot = np.zeros((3,1))
    
    with open(boot_parfit_file, 'r') as f:
        j = 0
        for line in f:
            i = 0
            boot_bestfit = np.zeros(5)
            line_element = line.strip().split("    ")
            boot_model_name = line_element[0][1:-1]
            for element in line_element[1:]:
                if "[" in element or "]" in element or "," in element:
                    parameter_bestfit = np.array(element.replace('[',
                            '').replace(']', '').split(','), dtype=np.float)
                    boot_bestfit = np.vstack((boot_bestfit, 
                                              parameter_bestfit))
                    i += 1  ## Loops through untied parameters
            if np.isnan(boot_bestfit).any():
#                 print "It's nan:, %d" % j
                pass
            else:
                all_boot_bestfits = np.dstack((all_boot_bestfits,
                        boot_bestfit[1:,]))
                max_phase_boot = []
                min_phase_boot = []
                for k in range(0,3):
                    this_boot = np.array([boot_bestfit[k+1,0], 
                                     boot_bestfit[k+1,1], 
                                     boot_bestfit[k+1,2], 
                                     boot_bestfit[k+1,3], 
                                     boot_bestfit[k+1,4]])
                    func = fit_function(t_bins, this_boot)
                    max_phase_boot.append(t_bins[np.argmax(func)])
                    min_phase_boot.append(t_bins[np.argmin(func)])
                
                all_max_phase_boot = np.hstack((all_max_phase_boot, np.reshape(np.array(max_phase_boot), (3,1))))
                all_min_phase_boot = np.hstack((all_min_phase_boot, np.reshape(np.array(min_phase_boot), (3,1))))
                
            j += 1  ## Loops through bootstraps

    ## Gets rid of the initial zeros
    all_boot_bestfits = all_boot_bestfits[:,:,1:] 
    all_max_phase_boot = all_max_phase_boot[:,1:]
    all_min_phase_boot = all_min_phase_boot[:,1:]
    
    ## Normalize phases to 1
    all_boot_bestfits[:,1,:] /= (2.0 * np.pi)
    all_boot_bestfits[:,3,:] /= (2.0 * np.pi)

    print "Shape of all boot bestfits:", np.shape(all_boot_bestfits)
    print "(spec parameter, funcfit parameter, bootstrap)"
    par_name = ["\Gamma", "\Fscatt", third]
    
    ##########
    
    data_bestfit_file = ES_BASEDIR + "/out_es/" + prefix + \
            "/" + prefix + "_" + day + "_" + fit_specifier +\
            "_final_bestfit.txt"
    
    assert os.path.isfile(data_bestfit_file), \
            "ERROR: File does not exist: %s" % data_bestfit_file
        
    with open(data_bestfit_file, 'r') as f:
        for line in f:
            data_bestfit = np.zeros(5)
            line_element = line.strip().split("    ")
            data_model_name = line_element[0][1:-1]
            for element in line_element[1:]:
                if "[" in element or "]" in element or "," in element:
                    parameter_bestfit = np.array(element.replace('[',
                            '').replace(']', '').split(','), dtype=np.float)
                    data_bestfit = np.vstack((data_bestfit, 
                                              parameter_bestfit))

    ## Gets rid of initial zeros
    data_bestfit = data_bestfit[1:,]  
    ## Normalize phases to 1
    data_bestfit[:,1] /= (2.0 * np.pi)
    data_bestfit[:,3] /= (2.0 * np.pi)
    
    print "\n\nShape of data bestfit:", np.shape(data_bestfit)
    maxval_phase = []
    minval_phase = []

    print "\midrule"
    for i in range(0,3):
        this_data = np.array([data_bestfit[i,0], 
                             data_bestfit[i,1]*2*np.pi, 
                             data_bestfit[i,2], 
                             data_bestfit[i,3]*2*np.pi, 
                             data_bestfit[i,4]])
        func = fit_function(t_bins, this_data)
        maxval_phase.append(t_bins[np.argmax(func)])
        minval_phase.append(t_bins[np.argmin(func)])
        
#         print " & $%s$ & $%.4f\pm%.4f$ & $%.3f\pm%.3f$ & $%.3f\pm%.3f$ & $%.3f\pm%.3f$ & $%.3f\pm%.3f$ & $%.3f\pm%.3f$ \\\\" % \
#                 (par_name[i], data_bestfit[i,0], np.sqrt(np.var(all_boot_bestfits[i,0,:], ddof=1)),
#                 data_bestfit[i,1], np.sqrt(np.var(all_boot_bestfits[i,1,:], ddof=1)),
#                 data_bestfit[i,2], np.sqrt(np.var(all_boot_bestfits[i,2,:], ddof=1)),
#                 data_bestfit[i,3], np.sqrt(np.var(all_boot_bestfits[i,3,:], ddof=1)),
#                 data_bestfit[i,4], np.sqrt(np.var(all_boot_bestfits[i,4,:], ddof=1)),
#                 t_bins[np.argmax(func)], np.sqrt(np.var(all_max_phase_boot[i,:], ddof=1)))
#         print np.var(func)
        print np.sqrt(np.var(func))
    
    print "\n Phase diffs from maxes"
    print "FS - G: $%.3f \pm %.3f$" % (maxval_phase[1] - maxval_phase[0], np.sqrt(np.var(all_max_phase_boot[1,:], ddof=1)) + np.sqrt(np.var(all_max_phase_boot[0,:], ddof=1)))
    print "FS - bb: $%.2f \pm %.2f$" % (maxval_phase[1] - maxval_phase[2], np.sqrt(np.var(all_max_phase_boot[1,:], ddof=1)) + np.sqrt(np.var(all_max_phase_boot[2,:], ddof=1)))
    print "Only for pBB: FS max - BB min: $%.2f \pm %.2f$" % ( maxval_phase[1] - minval_phase[2], np.sqrt(np.var(all_min_phase_boot[i,:], ddof=1)))

**Fit function:** p0 sin(2 pi t + p1) + p2 sin(4 pi t + p3) + p4

# 1 BB

In [9]:
fit_specifier = "1BB-FS-G-Tin-fzs-fzNbb"
get_funcfit_pars(fit_specifier, "\Tdisk")

Shape of all boot bestfits: (3, 5, 5532)
(spec parameter, funcfit parameter, bootstrap)


Shape of data bestfit: (3, 5)
\midrule
0.105282318079
0.0380752260354
0.00293991188555

 Phase diffs from maxes
FS - G: $0.005 \pm 0.015$
FS - bb: $0.32 \pm 0.02$
Only for pBB: FS max - BB min: $-0.10 \pm 0.03$


# 2 BB

In [22]:
fit_specifier = "2BB-FS-G-kT-fzs-fzNbb8857-2"
get_funcfit_pars(fit_specifier, "\Tbb")

Shape of all boot bestfits: (3, 5, 5504)
(spec parameter, funcfit parameter, bootstrap)


Shape of data bestfit: (3, 5)
\midrule
 & $\Gamma$ & $0.1738\pm0.0155$ & $-0.141\pm0.005$ & $0.014\pm0.011$ & $0.119\pm0.206$ & $2.438\pm0.011$ & $0.417\pm0.007$ \\
 & $\Fscatt$ & $0.0588\pm0.0030$ & $-0.189\pm0.002$ & $-0.007\pm0.001$ & $-0.108\pm0.008$ & $0.180\pm0.003$ & $0.436\pm0.002$ \\
 & $\Tbb$ & $0.0060\pm0.0006$ & $0.118\pm0.013$ & $-0.001\pm0.001$ & $0.292\pm0.222$ & $0.493\pm0.002$ & $0.155\pm0.019$ \\

 Phase diffs from maxes
FS - G: $0.019 \pm 0.009$
FS - bb: $0.28 \pm 0.02$
Only for pBB: FS max - BB min: $-0.17 \pm 0.03$


# pBB

In [23]:
fit_specifier = "pBB-FS-G-p-fzs-fzNbb"
get_funcfit_pars(fit_specifier, "p")

Shape of all boot bestfits: (3, 5, 5530)
(spec parameter, funcfit parameter, bootstrap)


Shape of data bestfit: (3, 5)
\midrule
 & $\Gamma$ & $0.1706\pm0.0142$ & $-0.149\pm0.006$ & $0.015\pm0.012$ & $0.125\pm0.205$ & $2.439\pm0.011$ & $0.425\pm0.008$ \\
 & $\Fscatt$ & $0.0465\pm0.0021$ & $-0.200\pm0.003$ & $-0.006\pm0.001$ & $-0.129\pm0.013$ & $0.131\pm0.001$ & $0.446\pm0.004$ \\
 & $p$ & $-0.0045\pm0.0005$ & $0.168\pm0.016$ & $-0.001\pm0.001$ & $-0.189\pm0.545$ & $0.507\pm0.002$ & $0.547\pm0.039$ \\

 Phase diffs from maxes
FS - G: $0.021 \pm 0.012$
FS - bb: $-0.10 \pm 0.04$
Only for pBB: FS max - BB min: $0.33 \pm 0.02$
